Task 1

In [166]:
from bs4 import BeautifulSoup
import requests
import csv
import json
import re

In [167]:
url = "https://www.baraasallout.com/test.html"
page = requests.get(url)

soup = BeautifulSoup(page.text,"html")

In [168]:
headers=soup.findAll(["h1","h2"])
paragraphs_and_lists = soup.findAll(["p","li"])

In [169]:
with open("Extract_Text_Data.csv", "w", encoding="utf-8", newline="") as csvFile:  
    writer = csv.writer(csvFile)
    writer.writerow(["type", "content"]) 
    
   
    for header in headers:
        writer.writerow(["header", header.text.strip()])
    
    
    for item in paragraphs_and_lists:
        if item.name == "p":  
            writer.writerow(["paragraph", item.text.strip()]) 
        else:
            writer.writerow(["list item", item.text.strip()])  


Task 2 

In [170]:
rows = soup.findAll("tr")


with open("Extract_Table_Data.csv", "w", encoding="utf-8", newline="") as table:
    writer = csv.writer(table)

   
    for row in rows:
       
        cells = row.find_all(["td", "th"])
        row_data = [cell.text.strip() for cell in cells]  
        writer.writerow(row_data)

Task 3

In [171]:
book_cards = soup.find_all('div', attrs={"style": lambda value: value and 'border' in value and 'padding' in value})
books_info = []

def clean_text(text):
    text = text.replace("\u00a3", "").replace("\u2714", "").strip()
    text = re.sub(r'[^\w\s]', '', text)
    return text

for card in book_cards:
    title_tag = card.find('strong')
    book_title = clean_text(title_tag.text) if title_tag else "Unknown"

    price_tag = card.find('p', attrs={"style": lambda style: style and "color: green" in style})
    price = clean_text(price_tag.text) if price_tag else "N/A"

    stock_info = card.find_all('p')[-1].text if card.find_all('p') else "N/A"
    stock_info_clean = clean_text(stock_info)

    btn = card.find('button')
    button_text = clean_text(btn.text) if btn else "Add to Basket"

    books_info.append({
        "Book Title": book_title,
        "Price": price,
        "Stock Availability": stock_info_clean,
        "Button Text": button_text
    })

with open('Product_Information.json', 'w', encoding="utf-8") as bookfile:
    json.dump(books_info, bookfile, indent=4)


Task 4

In [172]:

form_data = []

forms = soup.find_all('form')

for form in forms:
    inputs = form.find_all('input')
    for input_tag in inputs:
        field_info = {
            "Field Name": input_tag.get('name', "N/A"),
            "Input Type": input_tag.get('type', "N/A"),
            "Default Value": input_tag.get('value', None)
        }
        form_data.append(field_info)

with open('Form_Input_Data.json', 'w', encoding="utf-8") as output_file:
    json.dump(form_data, output_file, indent=4)



Task 5

In [173]:
links_data = []
for a_tag in soup.find_all('a', href=True):
    link = {
        "text": a_tag.text,
        "href": a_tag['href']
    }
    links_data.append(link)


iframe_tag = soup.find('iframe', src=True)
video_data = {}
if iframe_tag:
    video_data['src'] = iframe_tag['src']


output_data = {
    "hyperlinks": links_data,
    "video_link": video_data
}


with open('extracted_links_data.json', 'w') as file:
    json.dump(output_data, file, indent=4)

Task 6

In [174]:
products_data = []
for product in soup.find_all('div', class_='product-card'):
    product_id = product.get('data-id')
    name_tag = product.find('p', class_='name')
    price_tag = product.find('p', class_='price', style='display: none;')
    colors_tag = product.find('p', class_='colors')

    name = name_tag.text if name_tag else 'N/A'
    price = price_tag.text if price_tag else 'N/A'
    colors = colors_tag.text.replace('Available colors: ', '') if colors_tag else 'N/A'

    product_info = {
        'id': product_id,
        'name': name,
        'price': price,
        'colors': colors
    }
    products_data.append(product_info)

with open('extracted_products_data.json', 'w') as file:
    json.dump(products_data, file, indent=4)
